# Fore-01 Generate basic query

El objetivo de este notebook es testear la query definida para descargar los datos básicos necesarios para los experimentos de forecasting de multiwarehouse. Específicamente nos interesa obtener todas las compras de un sitio dado (por ejemplo, MLB) y en un rango de fechas, y de ellas el día y hora de pago de la compra, el item comprado, la cantidad de items iguales y el destino del envío (si la compra se realizó con Mercado Envíos, de lo contrario, el campo queda nulo).

## Importaciones y definiciones

In [1]:
import os

from datetime import datetime
from pathlib import Path

from ml_forecasting.etl.base import ExperimentDataSource
from ml_forecasting.etl.dataset import DataPreparation
from ml_forecasting.config import CONFIGS_BY_SITE, train_dtypes

In [2]:
# parameters
data_root = Path.home().joinpath('/alloc/data/fury_ml-forecasting/etl/raw_data/')
site = 'MLB'
teradata_user = os.environ['SECRET_TERADATA_USER']
teradata_pass = os.environ['SECRET_TERADATA_PASS']

## Desarrollo

En primer lugar definimos el período de tiempo de compras que queremos descargar y chequeamos si el path donde vamos a guardar la información existe:

In [3]:
start_download = datetime.strptime('2019-09-07', '%Y-%m-%d')
end_download = datetime.strptime('2019-09-14', '%Y-%m-%d')

In [4]:
data_root.mkdir(exist_ok=True)

Descargamos los datos empleando la query definida en `ml_forecasting.etl.base`:

In [5]:
source = ExperimentDataSource(teradata_user, teradata_pass, data_root, site)
files = source.csvfiles(start_download, end_download, delta=(end_download - start_download).days)

In [6]:
for f in files:
    print('*', f)

* /alloc/data/fury_ml-forecasting/etl/raw_data/experimentdatasource-MLB-20190907-20190914/Fore_MLB_20190907.csv


Tenemos todos los datos guardados en el archivo, ahora vamos a llamar a `DataPreparation` del módulo `ml_forecasting.etl.dataset` para cargar y preparar el dataframe:

In [7]:
prep = DataPreparation(CONFIGS_BY_SITE[site])

In [8]:
df = prep.load(files, cols=train_dtypes(CONFIGS_BY_SITE[site]))

Veamos el tamaño y un ejemplo del dataframe resultante:

In [9]:
df.shape

(3392551, 9)

In [10]:
df.sample(5)

,ord_order_id,ite_item_id,tim_day_winning_date,tim_time_winning_date,bid_quantity_ok,shp_shipment_id,receiver_shp_add_city_id,receiver_shp_add_state_id,receiver_shp_add_zip_code
1649649,2.143908e+09,1.158212e+09,2019-09-10,142828,1,28096206293,TUxCQ0ZPUjM5MmFh,BR-CE,60181825
392895,2.141575e+09,1.237951e+09,2019-09-10,75401,1,28094272476,BR-SP-44,BR-SP,04080004
2485686,2.141060e+09,1.183554e+09,2019-09-07,202421,1,28093802580,TUxCQ0lTUjZmODQ5,BR-GO,76205000
3353072,2.146044e+09,1.221209e+09,2019-09-12,125058,1,NaN,NaN,NaN,NaN
2240736,2.141878e+09,8.521760e+08,2019-09-08,214242,1,28094501250,TUxCQ1RBUTZjYjVk,BR-RS,95860000


Por último, podemos comparar la cantidad de SI (items vendidos) que obtenemos a partir de la query generada con los datos registrados en el [dashboard de PortalBI](http://portalbi.ml.com/#/business_unit/2/dashboards/67) (en este caso, 4021412 SI):

In [11]:
SI_query = df.bid_quantity_ok.sum()
print("Cantidad de SI descargados en la query: {}".format(SI_query))

Cantidad de SI descargados en la query: 4021709


In [12]:
SI_BI = 4021412
print("Porcentaje de diferencia entre la query y el dashboard de BI: {:.3f}%".format((SI_query-SI_BI)/SI_BI*100))

Porcentaje de diferencia entre la query y el dashboard de BI: 0.007%


Podemos ver que la diferencia es menor al 0.01%, por lo cual asumimos que la query generada es correcta y se obtienen todas las compras realizadas en el período de interés.

## Conclusión

El objetivo de este notebook era testear la query definida para descargar los datos básicos necesarios para los experimentos de forecasting de multiwarehouse. Utilizando la query definida en `ml_forecasting.etl.base` pudimos descargar los datos de interés de las compras de MLB para un rango de fechas (fecha de compra, item_id, número de items en la compra y dirección de destino del envío). Por otra parte, con la clase `DataPreparation` definida en `ml_forecasting.etl.dataset` pudimos cargar y preparar el dataframe. De esta manera, seteamos las bases (query y preparación de datos) para poder obtener los datos a utilizar en los futuros análisis de forecasting.